In [383]:
using ITensors

In [384]:
s = siteinds("S=1/2", 4)
mps = randomMPS(ComplexF64, s; linkdims=4);

In [385]:
struct PState
    pstate::MPS
    label::Int
end

In [386]:
function complex_feature_map(x::Float64)
    s1 = exp(1im * (3π/2) * x) * cospi(0.5 * x)
    s2 = exp(-1im * (2π/2) * x) * sinpi(0.5 * x)
    return [s1, s2]
end

complex_feature_map (generic function with 1 method)

In [387]:
function generate_training_data(samples_per_class::Int)

    class_A_samples = zeros(samples_per_class, 4)
    class_B_samples = ones(samples_per_class, 4)
    all_samples = vcat(class_A_samples, class_B_samples)
    all_labels = Int.(vcat(zeros(size(class_A_samples)[1]), ones(size(class_B_samples)[1])))

    return all_samples, all_labels

end

generate_training_data (generic function with 1 method)

In [388]:
function sample_to_product_state(sample::Vector, site_inds::Vector{Index{Int64}})
    n_sites = length(site_inds)
    product_state = MPS(ComplexF64, site_inds; linkdims=1)
    for j=1:n_sites
        T = ITensor(site_inds[j])
        zero_state, one_state = complex_feature_map(sample[j])
        T[1] = zero_state
        T[2] = one_state
        product_state[j] = T 
    end
    return product_state
end

sample_to_product_state (generic function with 1 method)

In [389]:
function dataset_to_product_state(dataset::Matrix, labels::Vector, sites::Vector{Index{Int64}})

    all_product_states = Vector{PState}(undef, size(dataset)[1])
    for p=1:length(all_product_states)
        sample_pstate = sample_to_product_state(dataset[p, :], sites)
        sample_label = labels[p]
        product_state = PState(sample_pstate, sample_label)
        all_product_states[p] = product_state
    end

    return all_product_states

end

dataset_to_product_state (generic function with 1 method)

In [390]:
function loss_and_grad(B::ITensor, mps::MPS, product_state::PState, lid, rid)
    # for a single sample
    ps = product_state.pstate
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    diff_sq = (norm(yhat[] - y))^2
    loss = 0.5 * diff_sq

    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    return loss, grad
end

loss_and_grad (generic function with 1 method)

In [391]:
all_samples, all_labels = generate_training_data(100)
all_pstates = dataset_to_product_state(all_samples, all_labels, s);

In [392]:
BT = mps[1] * mps[2]

ITensor ord=3 (dim=2|id=413|"S=1/2,Site,n=1") (dim=2|id=654|"S=1/2,Site,n=2") (dim=4|id=630|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [393]:
function update(n_iters, BT, lr=0.8)
    BT_new = BT
    for iter in 1:n_iters
        loss_total, grad_total = loss_and_grad(BT_new, mps, all_pstates[1], 1, 2)
        for ps in 2:200
            loss_val, grad_val = loss_and_grad(BT_new, mps, all_pstates[ps], 1, 2)
            loss_total += loss_val
            grad_total += grad_val
        end
        loss_final = loss_total / 200
        grad_final = grad_total ./ 200
        println("Loss: $loss_final")
        BT_new = BT_new - lr * grad_final
    end
    return BT_new
end

update (generic function with 4 methods)

In [394]:
BT_new = update(100, BT)

Loss: 0.22111206441937317
Loss: 0.14151172122839858
Loss: 0.09056750158617526
Loss: 0.05796320101515235
Loss: 0.037096448649697425
Loss: 0.023741727135806343
Loss: 0.015194705366916101
Loss: 0.009724611434826301
Loss: 0.006223751318288828
Loss: 0.003983200843704847
Loss: 0.002549248539971098
Loss: 0.0016315190655815017
Loss: 0.0010441722019721676
Loss: 0.0006682702092621854
Loss: 0.00042769293392779607
Loss: 0.0002737234777137891
Loss: 0.0001751830257368244


Loss: 0.00011211713647156717
Loss: 7.175496734180462e-5
Loss: 4.592317909875522e-5
Loss: 2.9390834623202773e-5
Loss: 1.8810134158849748e-5
Loss: 1.2038485861663824e-5
Loss: 7.70463095146495e-6
Loss: 4.930963808937342e-6
Loss: 3.155816837719873e-6
Loss: 2.0197227761408306e-6


Loss: 1.2926225767300586e-6
Loss: 8.272784491072979e-7
Loss: 5.294582074286822e-7
Loss: 3.3885325275435726e-7
Loss: 2.1686608176278804e-7
Loss: 1.3879429232816017e-7
Loss: 8.882834709006783e-8
Loss: 5.68501421376352e-8
Loss: 3.638409096808426e-8
Loss: 2.32858182195743e-8
Loss: 1.4902923660537915e-8
Loss: 9.53787114274787e-9
Loss: 6.104237531350287e-9
Loss: 3.9067120200707954e-9
Loss: 2.5002956928330863e-9
Loss: 1.600189243417138e-9
Loss: 1.024121115788959e-9
Loss: 6.554375141034092e-10
Loss: 4.194800090279069e-10


Loss: 2.684672057787918e-10
Loss: 1.718190116967943e-10
Loss: 1.0996416748569571e-10
Loss: 7.037706719084906e-11
Loss: 4.504132300201766e-11
Loss: 2.8826446721384438e-11
Loss: 1.8448925902270965e-11
Loss: 1.180731257732395e-11
Loss: 7.556680048976928e-12
Loss: 4.836275231753962e-12
Loss: 3.0952161480284025e-12
Loss: 1.9809383348217e-12
Loss: 1.2678005343111612e-12
Loss: 8.113923421046677e-13
Loss: 5.192910988100631e-13
Loss: 3.3234630332301023e-13
Loss: 2.1270163407663198e-13
Loss: 1.3612904585551429e-13
Loss: 8.712258934831673e-14
Loss: 5.575845717300927e-14
Loss: 3.5685412589934756e-14
Loss: 2.2838664067949886e-14
Loss: 1.4616744996059634e-14
Loss: 9.354716805720684e-15
Loss: 5.987018750934801e-15
Loss: 3.8316920029725835e-15
Loss: 2.4522828871378024e-15
Loss: 1.5694610419443923e-15
Loss: 1.0044550657520117e-15
Loss: 6.428512435831831e-16
Loss: 4.114247947291188e-16
Loss: 2.633118685963916e-16
Loss: 1.685195969495301e-16
Loss: 1.0785254244247427e-16
Loss: 6.902562755388246e-17
Loss: 

Loss: 2.8272896722418497e-17
Loss: 1.8094654254386602e-17
Loss: 1.1580578670633782e-17
Loss: 7.411569998877063e-18
Loss: 4.7434048899788125e-18
Loss: 3.035779000407016e-18
Loss: 1.9428985249168306e-18
Loss: 1.243455085541141e-18
Loss: 7.958111414083749e-19
Loss: 5.093191746516724e-19
Loss: 3.259642369755347e-19
Loss: 2.086171210520079e-19
Loss: 1.3351499593636543e-19
Loss: 8.54495926915212e-20
Loss: 5.468772408227357e-20
Loss: 3.50001398058747e-20
Loss: 2.240009779081247e-20
Loss: 1.4336068091412902e-20


ITensor ord=3 (dim=2|id=413|"S=1/2,Site,n=1") (dim=2|id=654|"S=1/2,Site,n=2") (dim=4|id=630|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [395]:
test_ps = all_pstates[150].pstate

MPS
[1] ((dim=2|id=413|"S=1/2,Site,n=1"),)
[2] ((dim=2|id=654|"S=1/2,Site,n=2"),)
[3] ((dim=2|id=47|"S=1/2,Site,n=3"),)
[4] ((dim=2|id=893|"S=1/2,Site,n=4"),)


In [396]:
yhat = BT_new * test_ps[1] * test_ps[2] * mps[3] * test_ps[3] * mps[4] * test_ps[4]

ITensor ord=0
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [397]:
abs(yhat[])

0.9999999998233658

# Have a look at sites 2-3

In [398]:
function loss_and_grad23(B::ITensor, mps::MPS, product_state::PState, lid, rid)
    # for a single sample
    ps = product_state.pstate
    phi_tilde = ps[lid] * ps[rid] * mps[1] * ps[1] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    diff_sq = (norm(yhat[] - y))^2
    loss = 0.5 * diff_sq

    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    return loss, grad
end

loss_and_grad23 (generic function with 1 method)

In [399]:
B23 = mps[2] * mps[3]

ITensor ord=4 (dim=4|id=243|"Link,l=1") (dim=2|id=654|"S=1/2,Site,n=2") (dim=2|id=47|"S=1/2,Site,n=3") (dim=2|id=494|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [400]:
function update23(n_iters, BT, lr=0.8)
    BT_new = BT
    for iter in 1:n_iters
        loss_total, grad_total = loss_and_grad23(BT_new, mps, all_pstates[1], 2, 3)
        for ps in 2:200
            loss_val, grad_val = loss_and_grad23(BT_new, mps, all_pstates[ps], 2, 3)
            loss_total += loss_val
            grad_total += grad_val
        end
        loss_final = loss_total / 200
        grad_final = grad_total ./ 200
        println("Loss: $loss_final")
        BT_new = BT_new - lr * grad_final
    end
    return BT_new
end

update23 (generic function with 2 methods)

In [401]:
B23_new = update23(100, B23, 1.5)

Loss: 0.22111206441937312
Loss: 0.14308611987229525
Loss: 0.0926249493073572
Loss: 0.05998043752335591
Loss: 0.038855126268809895
Loss: 0.025179671469552675
Loss: 0.01632378722485342
Loss: 0.010586858413370844
Loss: 0.0068690212586875275
Loss: 0.0044587232854937115
Loss: 0.0028954793049482067
Loss: 0.0018811832933585371
Loss: 0.001222781620852799
Loss: 0.0007952071456622933
Loss: 0.0005174062286913837
Loss: 0.00033682896195876867
Loss: 0.00021939156603438867
Loss: 0.00014297806282795474
Loss: 9.323178912361633e-5
Loss: 6.082887471852783e-5
Loss: 3.9711160259727147e-5
Loss: 2.5940485593466037e-5
Loss: 1.6955549233612816e-5
Loss: 1.1089683182649807e-5
Loss: 7.257799494643711e-6
Loss: 4.753068861210859e-6


Loss: 3.1148061851265926e-6
Loss: 2.0425838076989216e-6
Loss: 1.3403693545063975e-6
Loss: 8.801735228167707e-7
Loss: 5.783815035653488e-7
Loss: 3.803342747532849e-7
Loss: 2.502786397188277e-7
Loss: 1.6481290661226497e-7
Loss: 1.0860984423915134e-7
Loss: 7.162401982479433e-8
Loss: 4.7267242327166145e-8
Loss: 3.121576684866569e-8
Loss: 2.0630016178114973e-8
Loss: 1.3643825231617394e-8
Loss: 9.029890953690969e-9
Loss: 5.980492741624633e-9
Loss: 3.963669485189509e-9
Loss: 2.6288232601354588e-9
Loss: 1.7447207294758195e-9
Loss: 1.1587446260877755e-9
Loss: 7.700929751574292e-10
Loss: 5.121394109796415e-10
Loss: 3.408148466522279e-10
Loss: 2.269496124591824e-10
Loss: 1.5122239004153503e-10
Loss: 1.0082617326027121e-10
Loss: 6.726599869042144e-11
Loss: 4.490321205641116e-11
Loss: 2.9992517598860155e-11
Loss: 2.0044543622536292e-11


Loss: 1.3403587690787093e-11
Loss: 8.967706238451437e-12
Loss: 6.003034130946385e-12
Loss: 4.020527261236598e-12
Loss: 2.6940866349360753e-12
Loss: 1.8061341940289843e-12
Loss: 1.2114121961142862e-12
Loss: 8.128883684847369e-13
Loss: 5.457082494386899e-13
Loss: 3.665003704341974e-13
Loss: 2.462443722258969e-13
Loss: 1.6551219121758095e-13
Loss: 1.1129081161954168e-13
Loss: 7.485972756559135e-14
Loss: 5.0372192941834945e-14
Loss: 3.3906373753230236e-14
Loss: 2.2830425827374093e-14
Loss: 1.537741374677964e-14
Loss: 1.0360573190446764e-14
Loss: 6.982487624913757e-15
Loss: 4.7071423498794484e-15
Loss: 3.174097458477373e-15
Loss: 2.1408887544438028e-15
Loss: 1.444355711643023e-15
Loss: 9.746661634273413e-16
Loss: 6.5786211816461e-16
Loss: 4.441266984116263e-16
Loss: 2.998940061440639e-16
Loss: 2.025413213542414e-16
Loss: 1.3681717701923569e-16
Loss: 9.24368362836947e-17
Loss: 6.24630863718102e-17


Loss: 4.2215539950630336e-17
Loss: 2.853569943575987e-17
Loss: 1.929162649654394e-17
Loss: 1.304398534087881e-17
Loss: 8.820841626860707e-18
Loss: 5.965752513077193e-18
Loss: 4.035276664445495e-18
Loss: 2.7298057674718317e-18
Loss: 1.846877445437956e-18
Loss: 1.249654571500108e-18
Loss: 8.456396565138734e-19
Loss: 5.722976507837429e-19


ITensor ord=4 (dim=4|id=243|"Link,l=1") (dim=2|id=654|"S=1/2,Site,n=2") (dim=2|id=47|"S=1/2,Site,n=3") (dim=2|id=494|"Link,l=3")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

# Sites 3-4

In [402]:
function loss_and_grad34(B::ITensor, mps::MPS, product_state::PState, lid, rid)
    # for a single sample
    ps = product_state.pstate
    phi_tilde = ps[lid] * ps[rid] * mps[1] * ps[1] * mps[2] * ps[2]
    yhat = B * phi_tilde
    y = product_state.label
    diff_sq = (norm(yhat[] - y))^2
    loss = 0.5 * diff_sq

    dP = yhat[] - y
    grad = 0.5 * dP * conj(phi_tilde)

    return loss, grad
end

loss_and_grad34 (generic function with 1 method)

In [403]:
function update34(n_iters, BT, lr=0.8)
    BT_new = BT
    for iter in 1:n_iters
        loss_total, grad_total = loss_and_grad34(BT_new, mps, all_pstates[1], 3, 4)
        for ps in 2:200
            loss_val, grad_val = loss_and_grad34(BT_new, mps, all_pstates[ps], 3, 4)
            loss_total += loss_val
            grad_total += grad_val
        end
        loss_final = loss_total / 200
        grad_final = grad_total ./ 200
        println("Loss: $loss_final")
        BT_new = BT_new - lr * grad_final
    end
    return BT_new
end

update34 (generic function with 2 methods)

In [404]:
B34 = mps[3] * mps[4]

ITensor ord=3 (dim=4|id=630|"Link,l=2") (dim=2|id=47|"S=1/2,Site,n=3") (dim=2|id=893|"S=1/2,Site,n=4")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [452]:
B34_new = update34(100, B34, 0.8)

Loss: 0.22111206441937312
Loss: 0.19745918593547532


Loss: 0.1763707810380555
Loss: 0.15756613801765854
Loss: 0.14079547735163567
Loss: 0.12583652762520464
Loss: 0.11249148166454638
Loss: 0.10058429057423243
Loss: 0.08995825808563648
Loss: 0.08047390181160656
Loss: 0.07200705172406853
Loss: 0.06444715947768065
Loss: 0.05769579514037102
Loss: 0.05166531050175977
Loss: 0.046277650449647954
Loss: 0.041463295965368685
Loss: 0.03716032411975203
Loss: 0.03331357207831597
Loss: 0.029873893569872693
Loss: 0.026797497557229214
Loss: 0.024045359990035917
Loss: 0.021582700534067713
Loss: 0.019378517072469528
Loss: 0.01740517157537963
Loss: 0.015638021646036518
Loss: 0.014055092683929824
Loss: 0.012636786167599208
Loss: 0.011365620059176951
Loss: 0.01022599777666322
Loss: 0.009204002574436784
Loss: 0.008287214523119533
Loss: 0.007464547591534039
Loss: 0.006726104610460502


Loss: 0.006063048144069958
Loss: 0.005467485513718805
Loss: 0.004932366413277982
Loss: 0.0044513917280517915
Loss: 0.004018932323014101
Loss: 0.0036299567027003616
Loss: 0.0032799665665334604
Loss: 0.0029649393913229257
Loss: 0.002681277268657568
Loss: 0.0024257613102449797
Loss: 0.0021955110101195842
Loss: 0.0019879480200999675
Loss: 0.0018007638548558275
Loss: 0.0016318910962819727
Loss: 0.0014794777143036372
Loss: 0.001341864163415679
Loss: 0.0012175629517670106
Loss: 0.001105240412961153
Loss: 0.0010037004404148332
Loss: 0.0009118699705070272
Loss: 0.0008287860242260496
Loss: 0.0007535841379045113
Loss: 0.0006854880322099384
Loss: 0.0006238003850857602
Loss: 0.0005678945890433646
Loss: 0.0005172073862899807
Loss: 0.0004712322868205043
Loss: 0.00042951368496288634
Loss: 0.0003916415990882147
Loss: 0.00035724696740409984
Loss: 0.00032599744005585814


Loss: 0.000297593614263383
Loss: 0.0002717656650115471
Loss: 0.00024827032896707573
Loss: 0.00022688820388496462
Loss: 0.00020742132985526187
Loss: 0.00018969102238161687
Loss: 0.00017353593052554103
Loss: 0.00015881029623891234
Loss: 0.00014538239358064832
Loss: 0.0001331331288062938
Loss: 0.00012195478436254585
Loss: 0.0001117498916396187
Loss: 0.00010243021895739255
Loss: 9.391586270821912e-5
Loss: 8.613443086916051e-5
Loss: 7.902030924677112e-5
Loss: 7.251400184332674e-5
Loss: 6.656153764846513e-5
Loss: 6.111393697641987e-5
Loss: 5.612673119744333e-5
Loss: 5.155953036186248e-5
Loss: 4.737563379541814e-5
Loss: 4.354167926234019e-5
Loss: 4.002732675504461e-5
Loss: 3.6804973382245014e-5
Loss: 3.38494961961153e-5
Loss: 3.11380201286651e-5
Loss: 2.8649708501908997e-5
Loss: 2.636557383960801e-5
Loss: 2.426830694360106e-5
Loss: 2.2342122408109487e-5
Loss: 2.0572618933538226e-5
Loss: 1.894665296960947e-5
Loss: 1.7452224368257153e-5


Loss: 1.607837286150869e-5
Loss: 1.4815084300249716e-5


ITensor ord=3 (dim=4|id=630|"Link,l=2") (dim=2|id=47|"S=1/2,Site,n=3") (dim=2|id=893|"S=1/2,Site,n=4")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [524]:
function loss_12(B::ITensor, mps::MPS, product_state::PState, lid, rid)
    # for a single sample
    ps = product_state.pstate
    phi_tilde = ps[lid] * ps[rid] * mps[3] * ps[3] * mps[4] * ps[4]
    yhat = B * phi_tilde
    y = product_state.label
    diff_sq = (norm(yhat[] - y))^2
    loss = 0.5 * diff_sq

    return loss
end

loss_12 (generic function with 1 method)

In [525]:
B12 = mps[1] * mps[2]

ITensor ord=3 (dim=2|id=413|"S=1/2,Site,n=1") (dim=2|id=654|"S=1/2,Site,n=2") (dim=4|id=630|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [523]:
@show B12

B12 = ITensor ord=3
Dim 1: (dim=2|id=413|"S=1/2,Site,n=1")
Dim 2: (dim=2|id=654|"S=1/2,Site,n=2")
Dim 3: (dim=4|id=630|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2×4
[:, :, 1] =
 -0.011176392050295785 + 0.28929905299377073im   -0.29638740064941244 + 0.027511573555099568im
   -0.1650554368853493 - 0.1472498089401875im   -0.045511318840067996 - 0.10340716952510831im

[:, :, 2] =
 -0.10236124902585572 - 0.18574652683142714im  0.011737452924189316 - 0.2310982094116902im
  0.18980037564578417 + 0.14911737704430958im   0.29202260075115893 + 0.01449551122110714im

[:, :, 3] =
 0.009793988674870463 + 0.07931547124422478im    0.1136404349518007 - 0.18557827996355278im
  -0.2071428197889323 - 0.03600714727478321im  0.056428415510414665 - 0.03660519639393042im

[:, :, 4] =
 0.18064919194914703 - 0.14875480729253135im  -0.30049228593767024 + 0.03480729885156643im
  -0.286330315551739 + 0.08954992397959359im   0.42589159858873926 - 0.05908588037528349im


ITensor ord=3 (dim=2|id=413|"S=1/2,Site,n=1") (dim=2|id=654|"S=1/2,Site,n=2") (dim=4|id=630|"Link,l=2")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [526]:
loss_12(B12, mps, all_pstates[1], 1, 2)

0.05677892532462777

In [537]:
using Optim
using LinearAlgebra

In [570]:
i = Index(2, "i")
j = Index(2, "j")

(dim=2|id=616|"j")

In [571]:
B = randomITensor(i, j)

ITensor ord=2 (dim=2|id=640|"i") (dim=2|id=616|"j")
NDTensors.Dense{Float64, Vector{Float64}}

In [572]:
@show B

B = ITensor ord=2
Dim 1: (dim=2|id=640|"i")
Dim 2: (dim=2|id=616|"j")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
  1.717582811835526   -0.27212221015524796
 -1.3245198574281647   0.36383599737900213


ITensor ord=2 (dim=2|id=640|"i") (dim=2|id=616|"j")
NDTensors.Dense{Float64, Vector{Float64}}

In [575]:
T = 5.0

5.0

In [573]:
function cost_function(A_elems)
    A = ITensor(i, j)
    A[1,1] = A_elems[1]
    A[1,2] = A_elems[2]
    A[2,1] = A_elems[3]
    A[2,2] = A_elems[4]

    C = A * B
    C_val = scalar(C)
    cost = (C_val - T)^2
    return cost
end


cost_function (generic function with 1 method)

In [574]:
initial_guess = [0.1, 0.1, 0.1, 0.1]

4-element Vector{Float64}:
 0.1
 0.1
 0.1
 0.1

In [578]:
result = optimize(cost_function, initial_guess, ConjugateGradient())

 * Status: success

 * Candidate solution
    Final objective value:     2.993380e-25

 * Found with
    Algorithm:     Conjugate Gradient

 * Convergence measures
    |x - x'|               = 1.73e+00 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.45e-01 ≰ 0.0e+00
    |f(x) - f(x')|         = 2.45e+01 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 8.19e+25 ≰ 0.0e+00
    |g(x)|                 = 1.88e-12 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    3
    ∇f(x) calls:   2


In [579]:
Optim.minimizer(result)

4-element Vector{Float64}:
  1.8318007663759728
 -0.1743748067709329
 -1.2354840817295067
  0.46684779018185407

In [580]:
final = ITensor(i, j)

ITensor ord=2 (dim=2|id=640|"i") (dim=2|id=616|"j")
NDTensors.EmptyStorage{NDTensors.EmptyNumber, NDTensors.Dense{NDTensors.EmptyNumber, Vector{NDTensors.EmptyNumber}}}

In [582]:
final[1, 1] = 1.8318007663759728
final[1,2] = -0.1743748067709329
final[2,1] = -1.2354840817295067
final[2,2] = 0.46684779018185407

0.46684779018185407

In [587]:
@show final * B

final * B = ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}
 0-dimensional
5.000000000000547


ITensor ord=0
NDTensors.Dense{Float64, Vector{Float64}}

In [588]:
i = Index(2, "i")
j = Index(2, "j")

(dim=2|id=868|"j")

In [589]:
B = randomITensor(ComplexF64, i, j)

ITensor ord=2 (dim=2|id=248|"i") (dim=2|id=868|"j")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [590]:
@show B

B = ITensor ord=2
Dim 1: (dim=2|id=248|"i")
Dim 2: (dim=2|id=868|"j")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}
 2×2
  0.6516366826050329 - 0.3387672264355376im  -0.3955437836195299 + 1.1126374155072278im
 -0.8214268778237057 + 0.9092882513835908im  -0.6943601670492309 + 0.058139028743604465im


ITensor ord=2 (dim=2|id=248|"i") (dim=2|id=868|"j")
NDTensors.Dense{ComplexF64, Vector{ComplexF64}}

In [591]:
T = 5.0 + 3.0im

5.0 + 3.0im